In [9]:
import bioread
import pandas as pd
acq_file = bioread.read_file(r'C:\Users\User\Desktop\TIM.acq')

channels_to_extract = ['Heart Rate', 'ECG - ECG100C', 'C5 - Expression' ]
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
df.head()

,Heart Rate,ECG - ECG100C,C5 - Expression
0,0.0,0.118256,0.0
1,0.0,0.119476,0.0
2,0.0,0.119934,0.0
3,0.0,0.121307,0.0
4,0.0,0.121460,0.0


In [8]:
acq_file.channels

[Channel EDA - EDA100C: 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel ECG - ECG100C: 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel EMG - EMG100C: 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Stim current - CBLCFMA - Current Feedba: 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 0): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 1): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 2): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 3): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 4): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 5): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 6): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel Digital (STP Input 7): 7904435 samples, 2000.0 samples/sec, loaded: True,
 Channel H